In [13]:
from datasets import load_dataset

In [14]:
rawsets = load_dataset('papluca/language-identification')
id2label = sorted(set(rawsets['train']['labels']))

Using custom data configuration papluca--language-identification-088edc04c6573c74
Reusing dataset csv (/home/datngo/.cache/huggingface/datasets/papluca___csv/papluca--language-identification-088edc04c6573c74/0.0.0/652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)


  0%|          | 0/3 [00:00<?, ?it/s]

In [15]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-multilingual-cased")

In [16]:
def preprocess_func(examples):
    tokenized_example = tokenizer(examples['text'], truncation=True)
    tokenized_example['labels'] = [id2label.index(l) for l in examples['labels']]
    return tokenized_example

In [17]:
tokenized_sets = rawsets.map(preprocess_func, batched=True)

  0%|          | 0/70 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/10 [00:00<?, ?ba/s]

In [18]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [19]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=len(id2label))

Some weights of the model checkpoint at distilbert-base-multilingual-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'pre_classifie

In [20]:
from kheiron import Trainer, TrainingOptions

options = TrainingOptions(task='text-classification',
                          train_batch_size=32,
                          eval_batch_size=32,
                          metric_for_best_model='macro_f1',
                          greater_is_better=True,)

In [21]:
trainer = Trainer(model=model,
                  args=options,
                  train_set=tokenized_sets['train'],
                  eval_set=tokenized_sets['test'],
                  collate_fn=data_collator)

In [23]:
trainer.train()

Wed, 07 Sep 2022 11:22:65 | Trainer | INFO : ****** Trainning Process ****** 
Wed, 07 Sep 2022 11:22:65 | Trainer | INFO :   Training task            = text-classification
Wed, 07 Sep 2022 11:22:65 | Trainer | INFO :   Number of samples        = 70000
Wed, 07 Sep 2022 11:22:65 | Trainer | INFO :   Number of epochs         = 5
Wed, 07 Sep 2022 11:22:65 | Trainer | INFO :   Train batch size         = 32
Wed, 07 Sep 2022 11:22:65 | Trainer | INFO :   Total optimization steps = 10940
Wed, 07 Sep 2022 11:22:65 | Trainer | INFO :   Evaluation strategy      = 1/epoch
  0%|          | 0/2188 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 26.00 MiB (GPU 0; 3.82 GiB total capacity; 2.30 GiB already allocated; 2.62 MiB free; 2.36 GiB reserved in total by PyTorch)

In [ ]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-multilingual-cased", num_labels=len(id2label))

In [ ]:
from kheiron import Trainer, TrainingOptions

options = TrainingOptions(task='text-classification',
                          train_batch_size=32,
                          eval_batch_size=32,
                          metric_for_best_model='macro_f1',
                          greater_is_better=True,)

In [ ]:
trainer = Trainer(model=model,
                  args=options,
                  train_set=tokenized_imdb['train'],
                  eval_set=tokenized_imdb['test'],
                  collate_fn=data_collator)

In [ ]:
trainer.train()